<a href="https://colab.research.google.com/github/davidho27941/ML_tutorial_notebook/blob/main/Build_model_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 基於PyTorch平台的基本模型建立

## 載入必要函式庫以及資料集

為了在PyTorch建立模型，我們需要導入以下函式庫：

* torch 
* torchvision

以及常用的資料處理函式庫：

* Numpy 
* Pandas

我們可以透過`torch.cuda.is_available()`來確認GPU是否可用，並選擇在CPU或是GPU上運行我們的程式。

In [ ]:
import torchvision 
import torch 
import numpy as np
import pandas as pd 

# GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('Device State:', device)


Device State: cpu


## 匯入MNIST資料集

在此範例中，我們一樣會使用MNIST資料集進行示範。
在載入時，我們需要使用`torchvision`的`datasets.MNIST`函式來抽取MNSIT資料集。並利用經由`torchvision.transforms.Compose`所整理的轉置流程對資料集進行轉置。

我們將分別導入訓練資料集以及測試資料集作為訓練以及測試使用。

In [ ]:
# Transform
# Step 1: Convert to Tensor
# Step 2: Normalize with mean = 0.5 and std= 0.5
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5,), (0.5,)),]
)

# Obtain MNIST training dataset.
# Transform is a custom functon define by user, which allow user to apply the transformation when loading dataset.
mnist_train = torchvision.datasets.MNIST(root='MNIST', 
                            download=True, 
                            train=True, 
                            transform=transform)
                            
# Obtain MNIST test dataset
mnist_test = torchvision.datasets.MNIST(root='MNIST', 
                            download=True, 
                            train=False, 
                            transform=transform)

## 將資料打包並載入

在我們利用`torchvision.datasets.MNIST`函式將資料抽出後，我們可以利用`torch.utils.data.DataLoader`將資料轉置並載入成一個物件。

在`DataLoader`函式中，可以宣告：

* batch_size: 宣告批次數量大小。
* shuffle: 宣告是否要重新排序。
* sampler: 宣告自定義採樣器。

> 詳細參數可至[官方文件](https://pytorch.org/docs/stable/data.html)做進一步了解。

In [ ]:
trainLoader = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True)
testLoader = torch.utils.data.DataLoader(mnist_test, batch_size=64, shuffle=False)

## 建立順序式模型

在PyTorch之中，建立簡單的順序式模型與`TensorFlow`的作法相當類似。在`TensorFlow`中，我們利用`tf.keras.Sequential`函式來建立順序式模型，而在`PyTorch`之中我們使用`torch.nn`下屬的`torch.nn.Sequential`函數來完成順序式模型的建立。

以下是利用`torch.nn.Sequential`建立一個多層感知器（MLP）的範例。我們在一個順序性模型之中建立了三層的全連結層（利用`nn.Linear`）並結合線性整流函數(ReLU)以及對數歸一化指數函數（LogSoftmax）作為激勵函數。在模型建立完成之後，我們利用`.to()`將模型送到指定的裝置上進行使用。


In [ ]:
model = torch.nn.Sequential(
            torch.nn.Linear(in_features=784, out_features=128),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=128, out_features=64),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=64, out_features=10),
            torch.nn.LogSoftmax(dim=1)
            ).to(device)
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
  (5): LogSoftmax(dim=1)
)


In [ ]:
#@title 設定各項超參數、損失函數以及優化器

EPOCHS = 10 #@param {type:"slider", min:1, max:100, setp: 1}
LR = 1e-2 #@param {type:"number"}
OPTIMIZER = 'adam' #@param ["adam", "SGD"] {type:"raw"}

criterion = torch.nn.CrossEntropyLoss()
if OPTIMIZER == 'adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
elif  OPTIMIZER == 'SGD':
    optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9)


## 建立訓練流程 

在建立好模型以及設定好各項參數之後，我們可以開始建立我們的訓練流程。
在原生`PyTorch`之中，並不存在類似`TensorFlow`的`model.fit()`功能，所以我們必須自行將資料導入至模型之中，並取的模型的輸出，最後自行處理向前/向後傳播的流程。這邊簡單的將訓練流程給條列化。

1. 取得資料
2. 利用`optimizer.zero_grad()`將優化器既存的梯度值清空
3. 將資料導入模型，並取得模型輸出
4. 利用損失函數計算loss，並利用`loss.backward()`進行反向傳播
5. 將新的梯度值利用`optimizer.step()`將參數進行更新

因為原生`PyTorch`並未提供計算準確率的功能，也並未提供類似`TensorFlow`中，`model.compile()`中的`metrics`參數直接指定想看到的參數。所以我們需要直接提取模型輸出以及資料標籤進行準確度的評估。


In [ ]:
for epoch in range(EPOCHS):
    # 定義變數用以儲存單次訓練週期中的loss以及acc值
    running_loss = list()
    running_acc = 0.0

    for times, data in enumerate(trainLoader):

        # 宣告模型訓練狀態
        model.train()

        # 取得資料，並將其傳遞至相應裝置
        inputs, labels = data[0].to(device), data[1].to(device)

        # 將影像維度改為第一維度長度為inputs.shape[0]，第二維度為所剩維度展平的長度
        inputs = inputs.view(inputs.shape[0], -1)

        # 將既存梯度歸零
        optimizer.zero_grad()

        # 將資料導入模型，並取得模型輸出
        outputs = model(inputs)

        # 將模型輸出轉為整數
        predicted = torch.max(outputs.data, 1)[1]
        
        # 利用損失函數計算loss
        loss = criterion(outputs, labels)

        # 進行反向傳播
        loss.backward()

        # 更新參數
        optimizer.step()

        # 紀錄週期內的損失值
        running_loss.append(loss.item())

        # 計算週期內正確預測的數量
        running_acc += (labels==predicted).sum().item()
    
    # 計算週期為單位的loss以及acc
    _epoch_loss = torch.tensor(running_loss).mean()
    _epoch_acc = running_acc/(len(trainLoader)*64)

    # 輸出資訊
    print(f"Epoch : {epoch+1}, Epoch loss: {_epoch_loss:.4f}, Epoch Acc: {_epoch_acc:.2f}")
print('Training Finished.')

Epoch : 1, Epoch loss: 0.3909, Epoch Acc: 0.88
Epoch : 2, Epoch loss: 0.2502, Epoch Acc: 0.93
Epoch : 3, Epoch loss: 0.2253, Epoch Acc: 0.93
Epoch : 4, Epoch loss: 0.2185, Epoch Acc: 0.94
Epoch : 5, Epoch loss: 0.2102, Epoch Acc: 0.94
Epoch : 6, Epoch loss: 0.2062, Epoch Acc: 0.94
Epoch : 7, Epoch loss: 0.1880, Epoch Acc: 0.95
Epoch : 8, Epoch loss: 0.1860, Epoch Acc: 0.95
Epoch : 9, Epoch loss: 0.1932, Epoch Acc: 0.95
Epoch : 10, Epoch loss: 0.1791, Epoch Acc: 0.95
Training Finished.


## 對模型進行驗證

## 建立自定義模型

在先前的部份，我們利用`torch.nn.Sequential`建立了順序式模型。對於機器學習，常態下我們需要建立各式各樣的自訂模型結構。在`PyTorch`中，我們可以建立自定義類別物件，繼承`torch.nn.Module`類別，並在此類別內建立我的自訂模型架構。



In [ ]:
class my_model(torch.nn.Module):
    # 對類別進行初始化
    def __init__(self):
        # 呼叫繼承對象的__init__函數
        super(my_model, self).__init__()
        
        # 建立模型
        self.main = torch.nn.Sequential(
            torch.nn.Linear(in_features=784, out_features=128),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=128, out_features=64),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=64, out_features=10),
            torch.nn.LogSoftmax(dim=1)
        )
    
    # 定義向前傳播的過程
    def forward(self, input):
        return self.main(input)
my_model = my_model()
my_model

my_model(
  (main): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
    (5): LogSoftmax(dim=1)
  )
)

除了上方的在類別中利用`Sequential`建立模型的方式，也可以透過以下方式自定義模型。

In [ ]:
import torch.nn.functional as F

class my_model_1(torch.nn.Module):
    # 對類別進行初始化
    def __init__(self):
        # 呼叫繼承對象的__init__函數
        super(my_model_1, self).__init__()
        self.dense_1 = torch.nn.Linear(in_features=784, out_features=128)
        self.dense_2 = torch.nn.Linear(in_features=128, out_features=64)
        self.dense_3 = torch.nn.Linear(in_features=64, out_features=10)

    
    # 定義向前傳播的過程
    def forward(self, input):
        input = self.dense_1(input)
        input = F.relu(input)
        input = self.dense_2(input)
        input = F.relu(input)
        input = self.dense_3(input)
        return F.log_softmax(input, dim = 1)

my_model_1 = my_model_1()
my_model_1

my_model_1(
  (dense_1): Linear(in_features=784, out_features=128, bias=True)
  (dense_2): Linear(in_features=128, out_features=64, bias=True)
  (dense_3): Linear(in_features=64, out_features=10, bias=True)
)

## 結語

在本範例中，我們示範了如何用`nn.Sequential()`函數以繼承`nn.Module`類別的自定義類別來建立一個簡單的機器學習模型。接下來我們將了解如何透過建立`Callback`物件以及利用`Tensorboard`來監控模型的訓練流程。